In [2]:
import os

import earthpy as et
import geopandas as gpd
import holoviews as hv
import hvplot as hv
import hvplot.pandas
import hvplot.xarray
import json
import laspy
import numpy as np
import rasterio
import rasterio.features
import rioxarray as rxr
import rioxarray.merge as rxrm
from scipy.ndimage import binary_opening, binary_closing
import shapely
from shapely.geometry import shape
import whitebox
import zipfile

from shapely.ops import unary_union

### Pseudocode for process

import project area shapefile

import LIDAR index grid

intersect index grid and project areas shapefile to identify tiles to download

for each project area:
* download tiles
* process tiles with LASTools into canopy height dem
* clip to project area
* merge if necessary

Need to install PDAL, this requires installing visual studio build tools:
    
https://visualstudio.microsoft.com/visual-cpp-build-tools/

Select "Desktop Development with C++" option and install

then run pip install pdal

may need to install cmake from here too:

https://cmake.org/download/

In [3]:
# Inport the LIDAR index grid and set up directories
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
project_dir = os.path.join(data_dir, "treebeard")
# Create the directory if it doesn't exist
os.makedirs(data_dir, exist_ok=True)

las_index_path = os.path.join(
    data_dir,
    'earthpy-downloads',
    'lidar_index_cspn_q2',
    'lidar_index_cspn_q2.shp'
)

# Download LIDAR index tiles
if not os.path.exists(las_index_path):
    las_index_url = ('https://gisdata.drcog.org:8443/geoserver/DRCOGPUB/'
             'ows?service=WFS&version=1.0.0&request=GetFeature&'
             'typeName=DRCOGPUB:lidar_index_cspn_q2&outputFormat=SHAPE-ZIP')

    las_index_shp = et.data.get_data(url=las_index_url)

las_index_gdf = (
    gpd.read_file(las_index_path).set_index('tile')
#    .loc[['N3W345']]
)

las_index_gdf = las_index_gdf.to_crs('EPSG:4269')
crs = las_index_gdf.crs

las_index_plot = las_index_gdf.hvplot(
    tiles = 'OSM',
    crs=las_index_gdf.crs,
    geo = True,
    line_color='black',
    line_width=2,
    fill_alpha=0
)
las_index_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [4]:
las_index_gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [5]:
# Open project areas shapefile and plot
proj_zip_path = '../assets/project_areas_merged.zip'

with zipfile.ZipFile(proj_zip_path, 'r') as zip_ref:
    temp_dir = '/tmp/extracted_shapefile'  # You can specify any temporary directory
    zip_ref.extractall(temp_dir)
    
extracted_shapefile_path = temp_dir + '/'

proj_area_gdf = gpd.read_file(extracted_shapefile_path)

proj_area_gdf = proj_area_gdf.to_crs("EPSG:4326")

proj_area_plot = proj_area_gdf.hvplot(
    x='x',
    y='y',
    aspect='equal',
    tiles='EsriImagery',
    geo=True,
    line_color='blue',
    line_width=2,
    fill_alpha=0
)

proj_area_plot


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [x,y]

In [6]:
# Identify the tiles that intersect each project area
select_tiles_gdf = gpd.sjoin(las_index_gdf, proj_area_gdf, how='inner', predicate='intersects')

select_tiles_gdf.reset_index(drop=False)
select_tiles_gdf.hvplot(
    x='x',
    y='y',
    aspect='equal',
    tiles='EsriImagery',
    geo=True,
    line_color='blue',
    line_width=2,
    fill_alpha=0
)

C:\Users\Pete\AppData\Local\Temp\ipykernel_32024\2138564570.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4269
Right CRS: EPSG:4326

  select_tiles_gdf = gpd.sjoin(las_index_gdf, proj_area_gdf, how='inner', predicate='intersects')


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [x,y]

In [7]:
select_tiles_gdf = select_tiles_gdf.reset_index(drop=False)

In [8]:
# Generate list of all tiles per project area
tiles_by_area = select_tiles_gdf.groupby('Proj_ID')['tile'].apply(list).reset_index()
tiles_by_area

,Proj_ID,tile
0,Conifer Hill,"[N4W399, N4W397, N4W389, N4W396, N4W388, N4W29..."
1,Unnamed 1,[N4W264]
2,Unnamed 2,"[N4W381, N4W391]"
3,Zumwinkel,[N4W351]


In [9]:
# Process LAS files to canopy using Whitebox
def convert_las_to_tif(input_las, output_tif, return_type):
    """
    Converts a LAS file to a GeoTIFF using WhiteboxTools, based on the specified return type.

    Parameters
    ----------
    input_las : str
        Path to the input LAS file.
    output_tif : str
        Path to save the output GeoTIFF file.
    return_type : str
        Type of returns to process. Must be either 'first' for first returns or 'ground' for ground returns.

    Raises
    ------
    ValueError
        If `return_type` is not 'first' or 'ground'.

    Notes
    -----
    This function uses WhiteboxTools' `lidar_idw_interpolation` method to perform the conversion.
    The interpolation method used is Inverse Distance Weighting (IDW) with a resolution of 1.

    Examples
    --------
    >>> convert_las_to_tif_whitebox('input.las', 'output_first_returns.tif', 'first')
    >>> convert_las_to_tif_whitebox('input.las', 'output_ground_returns.tif', 'ground')
    """
    wbt = whitebox.WhiteboxTools()
    
    if return_type == "first":
        wbt.lidar_idw_interpolation(
            i=input_las,
            output=output_tif,
            parameter="return_num",
            returns=1,
            resolution=1  # Adjust as needed
        )
    elif return_type == "ground":
        wbt.lidar_idw_interpolation(
            i=input_las,
            output=output_tif,
            parameter="classification",
            classification=2,
            resolution=1  # Adjust as needed
        )
    else:
        raise ValueError("Invalid return_type. Use 'first' or 'ground'.")

In [10]:
# Process tiles for each project area
# Generate a dictionary of canopy TIFs for each project area

las_root_url = 'https://lidararchive.s3.amazonaws.com/2020_CSPN_Q2/'
canopy_dict = {}
for index, row in tiles_by_area.iterrows():
    tiles = row['tile']
    proj_area_name = row['Proj_ID']
    sel_proj_area_gdf = proj_area_gdf[proj_area_gdf['Proj_ID'] == proj_area_name]
    # Download all tiles for project area, process, and clip/merge
    tile_agg = []
    print("Processing LIDAR for " + proj_area_name)
    for tile in tiles:
        file_name = tile + ".las"
        print("Processing LIDAR tile " + tile)
        tile_path = os.path.join(
            data_dir,
            'earthpy-downloads',
            file_name
        )
        download_url = las_root_url + tile + ".las"
        if not os.path.exists(tile_path):
            et.data.get_data(url=download_url)
        # PDAL is required for this step, see readme for install instructions

        # Output path for first returns DEM
        output_fr_tif = os.path.join(
            project_dir,
            tile +'_fr.tif'
        )
        if not os.path.exists(output_fr_tif):
            convert_las_to_tif(tile_path, output_fr_tif, "first")
        
        # Output path for ground DEM
        output_gr_tif = os.path.join(
            project_dir,
            tile +'_gr.tif'
        )
        if not os.path.exists(output_gr_tif):
            convert_las_to_tif(tile_path, output_gr_tif, "ground")
        
        # Process ground and first return data to canopy height
        fr_dem = rxr.open_rasterio(output_fr_tif)
        fr_dem = fr_dem.rio.reproject("EPSG:4326")

        gr_dem = rxr.open_rasterio(output_gr_tif)
        gr_dem = gr_dem.rio.reproject("EPSG:4326")
        gr_dem = gr_dem.rio.reproject_match(fr_dem)

        canopy_dem = fr_dem - gr_dem

        # canopy_dem = canopy_dem.where(canopy_dem >= 1, -9999)
        # canopy_dem = canopy_dem.where(canopy_dem <= 500, np.nan)
        # Set all values greater than 1 (canopy) to 1 and all values less than 1 (no canopy) to 0
        canopy_dem.values[canopy_dem < 1] = 0
        canopy_dem.values[canopy_dem > 1] = 1
        canopy_dem.name = tile + "_Canopy"
        canopy_dem = canopy_dem.round()
        tile_agg.append(canopy_dem)
    print("Merging LIDAR tiles for " + proj_area_name)
    # Merge all tiles that intersect with the project area and clip to project area
    canopy_merged = rxrm.merge_arrays(tile_agg).rio.clip(sel_proj_area_gdf.geometry)
    canopy_dict[proj_area_name] = canopy_merged        

Processing LIDAR for Conifer Hill
Processing LIDAR tile N4W399
Processing LIDAR tile N4W397
Processing LIDAR tile N4W389
Processing LIDAR tile N4W396
Processing LIDAR tile N4W388
Processing LIDAR tile N4W290
Processing LIDAR tile N4W398
Processing LIDAR tile N3W308
Processing LIDAR tile N3W306
Processing LIDAR tile N3W307
Merging LIDAR tiles for Conifer Hill
Processing LIDAR for Unnamed 1
Processing LIDAR tile N4W264
Merging LIDAR tiles for Unnamed 1
Processing LIDAR for Unnamed 2
Processing LIDAR tile N4W381
Processing LIDAR tile N4W391
Merging LIDAR tiles for Unnamed 2
Processing LIDAR for Zumwinkel
Processing LIDAR tile N4W351
Merging LIDAR tiles for Zumwinkel


In [11]:
# Export Zumwinkel canopy tif to repo
test = canopy_dict['Zumwinkel']

zumwinkel_path = "../notebooks/zumwinkel_canopy.tif"

if not os.path.exists(zumwinkel_path):
    test.rio.to_raster(zumwinkel_path, overwrite=True)

In [12]:
# Plot Zumwinkel canopy tif
zumwinkel_path = "../notebooks/zumwinkel_canopy.tif"

zumwinkel_lidar = rxr.open_rasterio(zumwinkel_path).rio.reproject("EPSG:4326")
zumwinkel_lidar.hvplot(
    height=600,
    width=600,
    geo=True,
    #rasterize=True,
    aspect='equal',
    kind='image',
    tiles = 'EsriImagery',
    alpha=0.5,
    title = "LIDAR Canopy Example",
    clabel= 'Height in feet',
    crs = canopy_dem.rio.crs
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'bcebd01d-d5b5-4116-928e-e3e650d60163': {'version…

In [13]:
# Clean up "noise" in raster

# Function to apply morphological operations on a rioxarray DataArray
def clean_raster_rioxarray(raster_xarray, operation='opening', structure_size=3):
    # Extract the numpy array from the xarray DataArray
    raster_data = raster_xarray.values

    # Ensure the raster_data is 2D (in case it's a single-band raster with an extra dimension)
    if raster_data.ndim == 3 and raster_data.shape[0] == 1:
        raster_data = raster_data[0, :, :]
    
    # Convert to binary (tree canopy is represented by 1, no canopy by 0)
    binary_raster = raster_data == 1

    # Define the structure for the morphological operation
    structure = np.ones((structure_size, structure_size), dtype=int)

    # Apply the chosen morphological operation
    if operation == 'opening':
        cleaned_raster = binary_opening(binary_raster, structure=structure)
    elif operation == 'closing':
        cleaned_raster = binary_closing(binary_raster, structure=structure)
    else:
        raise ValueError("Operation must be 'opening' or 'closing'")

    # Convert back to the original values (1 for canopy, 0 for no canopy)
    raster_data_cleaned = np.where(cleaned_raster, 1, 0)

    # Add back the extra dimension if the original data had it
    if raster_xarray.values.ndim == 3:
        raster_data_cleaned = np.expand_dims(raster_data_cleaned, axis=0)

    # Create a new xarray DataArray with the cleaned data, copying metadata from the original
    cleaned_raster_xarray = raster_xarray.copy(data=raster_data_cleaned)

    return cleaned_raster_xarray

operation = 'opening'  # Choose 'opening' or 'closing'
structure_size = 3  # Size of the structure element for morphological operations

zumwinkel_lidar_cleaned = clean_raster_rioxarray(zumwinkel_lidar, operation, structure_size)

In [14]:
lidar_plot = zumwinkel_lidar_cleaned.hvplot(
    height=600,
    width=600,
    geo=True,
    #rasterize=True,
    aspect='equal',
    kind='image',
    tiles = 'EsriImagery',
    alpha=0.5,
    title = "LIDAR Canopy Example",
    clabel= 'Height in feet',
    crs = canopy_dem.rio.crs
)

In [15]:
lidar_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'90446de0-d422-4913-a836-c14b35144f4b': {'version…

In [16]:
# Create a vector binary mask for canopy

# Export Zumwinkel canopy tif to repo
zumwinkel_path = "../notebooks/zumwinkel_clean_canopy.tif"
zumwinkel_lidar_cleaned = zumwinkel_lidar_cleaned.where(zumwinkel_lidar_cleaned != 1.7976931348623157e+308, np.nan)
zumwinkel_lidar_cleaned.rio.to_raster(zumwinkel_path, overwrite=True)

# Load the TIF file using rioxarray
binary_mask = zumwinkel_lidar_cleaned.squeeze()  # Assuming the data is in the first band

# Create a mask where cell values are 1
mask = binary_mask == 1

# Get the affine transform from the raster data
transform = binary_mask.rio.transform()

# Extract shapes (polygons) from the binary mask
shapes = rasterio.features.shapes(mask.astype(np.int16).values, transform=transform)
polygons = [shape(geom) for geom, value in shapes if value == 1]

# Create a GeoDataFrame from the polygons
canopy_gdf = gpd.GeoDataFrame({'geometry': polygons})

# gdf.hvplot(    
#     x='x',
#     y='y',
#     aspect='equal',
#     tiles='EsriImagery',
#     geo=True,
#     line_color='blue',
#     line_width=2,
#     #fill_alpha=0,
#     width = 600,
#     height=600,
#     color='blue'
# )


In [17]:
# Prep data for vector processing. Make sure CRS is set to coordinate system with correct units.

crs = zumwinkel_lidar_cleaned.rio.crs

if canopy_gdf.crs is None:
    canopy_gdf = canopy_gdf.set_crs(zumwinkel_lidar_cleaned.rio.crs)

# Define EPSG:6430 CRS
epsg_6430 = '6430'

# Reproject to EPSG:6430
canopy_gdf = canopy_gdf.to_crs(epsg=epsg_6430)

In [18]:
zumwinkel_boundary= proj_area_gdf[proj_area_gdf['Proj_ID'] == "Zumwinkel"]
zumwinkel_boundary = zumwinkel_boundary.to_crs("EPSG:6430")

In [19]:
# Method to process canopy gaps.

def process_canopy_areas(canopy_gdf, study_area, buffer_distance=5):
    """
    Processes canopy areas by buffering, dissolving, clipping, and exploding the geometries.
    Adds acreage and size category columns.

    Parameters
    ----------
    canopy_gdf : gpd.GeoDataFrame
        GeoDataFrame representing canopy areas.
    study_area : gpd.GeoDataFrame
        GeoDataFrame representing the boundary within which to clip the canopy areas.
    buffer_distance : float, optional
        The distance to buffer the canopy geometries. Default is 5 units.

    Returns
    -------
    exploded_gap_gdf : gpd.GeoDataFrame
        GeoDataFrame with exploded geometries representing non-tree canopy areas, including acreage and size category.
    """
    
    # Ensure input GeoDataFrames have CRS
    if canopy_gdf.crs is None or study_area.crs is None:
        raise ValueError("Input GeoDataFrames must have a CRS defined.")

    # Buffer the canopy geometries
    buffered_canopy = canopy_gdf.geometry.buffer(buffer_distance)

    # Create a new GeoDataFrame with the buffered geometries
    buffer_gdf = gpd.GeoDataFrame(geometry=buffered_canopy, crs=canopy_gdf.crs)

    # Dissolve the buffered geometries into a single MultiPolygon
    dissolved_canopy = unary_union(buffer_gdf.geometry)

    # Convert the dissolved canopy back to a GeoDataFrame
    dissolved_canopy_gdf = gpd.GeoDataFrame(geometry=[dissolved_canopy], crs=canopy_gdf.crs)

    # Clip the dissolved canopy with the study area
    clipped_buffer = gpd.overlay(dissolved_canopy_gdf, study_area, how='intersection')

    # Calculate the difference between the study area and the clipped buffer
    non_tree_canopy_gdf = gpd.overlay(study_area, clipped_buffer, how='difference')

    # Explode multipart polygon to prepare for area calculations
    exploded_gap_gdf = non_tree_canopy_gdf.explode(index_parts=True)

    # Reset the index to have a clean DataFrame
    exploded_gap_gdf.reset_index(drop=True, inplace=True)

    # Calculate the area in acres (1 acre = 43,560 square feet)
    exploded_gap_gdf['Acreage'] = exploded_gap_gdf.geometry.area / 43560

    # Define a function to categorize the gap size
    def categorize_gap_size(acres):
        if acres < 1/8:
            return '< 1/8 acre'
        elif 1/8 <= acres < 1/4:
            return '1/8 - 1/4 acre'
        elif 1/4 <= acres < 1/2:
            return '1/4 - 1/2 acre'
        elif 1/2 <= acres < 1:
            return '1/2 - 1 acre'
        else:
            return '> 1 acre'

    # Apply the categorization function to the Acreage column
    exploded_gap_gdf['Gap_Size_Category'] = exploded_gap_gdf['Acreage'].apply(categorize_gap_size)

    return exploded_gap_gdf

In [20]:
canopy_gaps_calced = process_canopy_areas(canopy_gdf, zumwinkel_boundary, buffer_distance=5)

In [21]:
canopy_gaps_calced.to_file('canopy_gaps_calced.shp')

C:\Users\Pete\AppData\Local\Temp\ipykernel_32024\2722419606.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  canopy_gaps_calced.to_file('canopy_gaps_calced.shp')
c:\Users\Pete\miniconda3\envs\treebeard\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Gap_Size_Category' to 'Gap_Size_C'
  ogr_write(


In [22]:
canopy_gaps_calced = canopy_gaps_calced.to_crs("EPSG:4326")
canopy_gaps_calced.hvplot(
    x='x',
    y='y',
    aspect='equal',
    geo=True,
    line_color='blue',
    line_width=2,
    #fill_alpha=0,
    width = 600,
    height=600,
    tiles = 'EsriImagery',
    title = "Processed Canopy Gaps from LIDAR"
)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [x,y]